In [1]:
# load env variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
from langchain_community.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
# create a tool first

@tool
def multiply(a:int,b:int)->int:
    """Multiply two numbers"""
    return a*b

In [4]:
multiply.invoke({"a":4,"b":5})

20

In [6]:
# now create a llm
llm = ChatGroq(model="llama-3.3-70b-versatile")

# and bind the llm to the multiply tool
llm_with_tool = llm.bind_tools([multiply])

In [7]:
# ask a simple question to the llm
response = llm_with_tool.invoke("Whats up?")
response.content

"I'm here to help with any questions or tasks you may have. What's on your mind?"

In [9]:
# then ask the llm to multiply two numbers
response = llm_with_tool.invoke("Can you multiply 7 and 8?")
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 7, 'b': 8},
  'id': '7kmwym03d',
  'type': 'tool_call'}]

In [10]:
# execute the tool with tool args
multiply.invoke(response.tool_calls[0]['args'])

56

In [11]:
# but if you execute the tool with the whole schema, you get ToolMessage
multiply.invoke(response.tool_calls[0])

ToolMessage(content='56', name='multiply', tool_call_id='7kmwym03d')

In [12]:
# now we will pass this toolmessage to llm, along with all previous messages, so that llm has a context
# create a HumanMessage
query = HumanMessage("Can you multiply 5 with 7 ?")

In [13]:
# append the message to a list, where you will maintain chat history
messages = [query]

# send the messages to llm
response = llm_with_tool.invoke(messages)

In [14]:
# now this response is an AIMessage
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '6skrenfhn', 'function': {'arguments': '{"a":5,"b":7}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 221, 'total_tokens': 240, 'completion_time': 0.065941927, 'completion_tokens_details': None, 'prompt_time': 0.012164576, 'prompt_tokens_details': None, 'queue_time': 0.056216984, 'total_time': 0.078106503}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd9dd-4340-7800-8c35-148ac1a12fe6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 7}, 'id': '6skrenfhn', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 221, 'output_tokens': 19, 'total_tokens': 240})

In [16]:
# append this AIMessage to the list
messages.append(response)

# now execute the tool, and get the tool message
tool_msg = multiply.invoke(response.tool_calls[0])

In [17]:
# append the tool msg to chat history as well
messages.append(tool_msg)

messages

[HumanMessage(content='Can you multiply 5 with 7 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '6skrenfhn', 'function': {'arguments': '{"a":5,"b":7}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 221, 'total_tokens': 240, 'completion_time': 0.065941927, 'completion_tokens_details': None, 'prompt_time': 0.012164576, 'prompt_tokens_details': None, 'queue_time': 0.056216984, 'total_time': 0.078106503}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd9dd-4340-7800-8c35-148ac1a12fe6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 7}, 'id': '6skrenfhn', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 221, 'output_tokens': 19, 'total_tokens': 240}),
 AIMessage(

In [19]:
# now pass the whole the llm, and get the reply from llm
reply = llm_with_tool.invoke(messages)

reply.content

'The result of multiplying 5 by 7 is 35.'

So step by step we did -> Tool creation , Tool binding , Tool calling , Tool execution